In [1]:
print("ping")

ping


In [2]:
# Core
from __future__ import absolute_import, division, print_function, unicode_literals

# Libraries
import numpy as np
import pyspark as ps
import pandas as pd
import matplotlib.pyplot as plot
from IPython.display import clear_output

import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

# Utils

from utils import Session

In [3]:
session = Session("TensorFlow")
spark = session.session
spark

In [5]:
spark.sparkContext.addFile(
    "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
spark.sparkContext.addFile(
    "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)

In [10]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [33]:
# Define the schema
from pyspark.sql.types import StructField,StringType,IntegerType,StructType, FloatType

schema = []
for column in CSV_COLUMN_NAMES:
    if column == 'Species':
        schema.append(StructField(column, IntegerType(), True))
    else:
        schema.append(StructField(column, FloatType(), True))

schema = StructType(fields=schema)


In [34]:
dftrain = spark.read.csv(
    ps.SparkFiles.get("iris_training.csv"), header = False, schema = schema
).toPandas().tail(-1)

dfeval = spark.read.csv(
    ps.SparkFiles.get("iris_test.csv"), header = False, schema = schema
).toPandas().tail(-1)

In [35]:
dftrain.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
1,6.4,2.8,5.6,2.2,2.0
2,5.0,2.3,3.3,1.0,1.0
3,4.9,2.5,4.5,1.7,2.0
4,4.9,3.1,1.5,0.1,0.0
5,5.7,3.8,1.7,0.3,0.0


In [36]:
train_y = dftrain.pop('Species').astype(int) # Remove the species column
test_y = dfeval.pop('Species').astype(int)

train_y.head()


1    2
2    1
3    2
4    0
5    0
Name: Species, dtype: int64

In [19]:
def make_input_function(
        features, 
        labels, 
        training=True, 
        batch_size=256):

    def input_fn():
        # Convert the inputs to a Dataset.
        dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

        # Shuffle and repeat if you are in training mode.
        if training:  # If training is True
            dataset = dataset.shuffle(1000).repeat() # Shuffle 1000 times 
        
        return dataset.batch(batch_size)
    return input_fn

# instead of calling lambda: input_fn(*args) we define the function wrapped.

In [37]:
# Feature columns describe how to use the input. We don't need the numerical of the previous input function because we're looking for the key values
my_feature_columns = []

for key in dftrain.keys(): #this code will loop through all of the keys in the dataset
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

clear_output()
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [38]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=len(SPECIES))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpb16n0w79', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [39]:
# monkey-patching AttributeError: module 'inspect' has no attribute 'getargspec'
import inspect

if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec


In [40]:
# 1) Training the DNN model
classifier.train(
    # input_fn=lambda: input_fn(train, train_y, training=True),
    input_fn = make_input_function(dftrain, train_y),
    steps=5000)
# We include a lambda to avoid creating an inner function previously
     

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpb16n0w79/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Us

In [41]:
# 1) Test the DNN model
eval_result = classifier.evaluate(
    # input_fn=lambda: input_fn(train, train_y, training=True),
    input_fn = make_input_function(dfeval, test_y, training=False))
# We include a lambda to avoid creating an inner function previously

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-07-31T16:07:54
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpb16n0w79/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.66886s
INFO:tensorflow:Finished evaluation at 2023-07-31-16:07:55
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8666667, average_loss = 0.4938505, global_step = 5000, loss = 0.4938505
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpb16n0w79/model.ckpt-5000

Test set accuracy: 0.867



In [43]:
# 3) Predict the Classification in DNN model
def predict_input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
# Here is some example input and expected classes you can try above
predict = {
    'SepalLength': [5.1, 5.9, 6.9],  # for ['Setosa', 'Versicolor', 'Virginica']
    'SepalWidth' : [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth' : [0.5, 1.5, 2.1],
}

predict_df = pd.DataFrame(
    data = [
        [5.1, 3.3, 1.7, 0.5],   # Setosa
        [5.9, 3.0, 4.2, 1.5],   # Versicolor
        [6.9, 3.1, 5.4, 2.1]    # Virginica
    ],
    columns=features
)


predictions = classifier.predict(input_fn=lambda: predict_input_fn(predict_df))

clear_output()

for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpb16n0w79/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 0.17967242, -1.5329056 , -3.0539646 ], dtype=float32), 'probabilities': array([0.8197971 , 0.14789148, 0.03231139], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" (82.0%)
{'logits': array([-3.8053029, -2.2266154, -2.402877 ], dtype=float32), 'probabilities': array([0.10087115, 0.4890826 , 0.41004616], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Versicolor" (48.9%)
{'logits': array([-5.5088744,